In [1]:
import sys
import torch
import numpy as np
#from torchani.models import ANI2x
#from openmmtorch import TorchForce
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from openmm.app import PDBFile

In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")

if cuda_available:
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
    current_gpu_name = torch.cuda.get_device_name(0)
    print(f"Current GPU Name: {current_gpu_name}")
    print("\nGPU Memory Usage:")
    print(f"  Allocated: {round(torch.cuda.memory_allocated(0) / 1024**3, 1)} GB")
    print(f"  Cached: {round(torch.cuda.memory_reserved(0) / 1024**3, 1)} GB")
else:
    print("PyTorch is running on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

CUDA Available: True
Number of GPUs: 1
Current GPU Name: NVIDIA GeForce RTX 5070 Laptop GPU

GPU Memory Usage:
  Allocated: 0.0 GB
  Cached: 0.0 GB

Using device: cuda


/home/haolan/anaconda3/envs/openmm_ml/lib/python3.11/site-packages/torch/cuda/__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_89 sm_90 compute_90.
If you want to use the NVIDIA GeForce RTX 5070 Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


In [ ]:
'''
from openff.toolkit.topology import Molecule
molecule=Molecule.from_file('bioactive_min.sdf')
positions=molecule.conformers[0].to_openmm()
from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)
forcefield = app.ForceField('amber14/tip3p.xml')
forcefield.registerTemplateGenerator(gaff.generator)
#
modeller = app.Modeller(molecule.to_topology().to_openmm(), positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*unit.nanometers)
#
system_1 = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
with open('gaff_system.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system_1))
with open('gaff_ligand_in_solvent.pdb', 'w') as output_pdb:
    PDBFile.writeFile(modeller.topology, modeller.positions, output_pdb)
'''

In [3]:
with open('gaff_system.xml', 'r') as f:
    system_1 = mm.XmlSerializer.deserialize(f.read())
pdb = PDBFile('gaff_ligand_in_solvent.pdb')
#system_1.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

In [4]:
chains = list(pdb.topology.chains())
print(chains)
ml_atoms = [atom.index for atom in chains[0].atoms()]
print(ml_atoms)

[<Chain 0>, <Chain 1>]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]


#ML/MM (solute use ANIx while solvents and intermolecular interactions use classic MM)

In [5]:
from openmmml import MLPotential # this module include functions of torchani and openmmtorch
potential = MLPotential('ani2x', implementation='nnpops')
mixed_system = potential.createMixedSystem(pdb.topology, system_1, ml_atoms)

/home/haolan/anaconda3/envs/openmm_ml/lib/python3.11/site-packages/torchani/__init__.py:59: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")


/home/haolan/anaconda3/envs/openmm_ml/lib/python3.11/site-packages/torchani/resources/


In [6]:
# Create an integrator with a time step of 0.5 fs (ensure NN potential gradient force precision and system stability)
temperature = 300 * unit.kelvin
frictionCoeff = 1 / unit.picosecond
timeStep = 0.5 * unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, frictionCoeff, timeStep)

# Create a simulation and set the initial positions and velocities
simulation = app.Simulation(pdb.topology, mixed_system, integrator)
print("Simulation is running on:", simulation.context.getPlatform().getName())
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(temperature)

# Minimize the system
simulation.minimizeEnergy()
with open(f'min_complex.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)

Simulation is running on: CUDA


: 

In [ ]:
# Configure a reporter to print to the console every 100 steps 
reporter = app.StateDataReporter(file=sys.stdout, reportInterval=100, step=True, time=True, potentialEnergy=True, temperature=True, speed=True)
simulation.reporters.append(reporter)
simulation.step(10000)